## VEGETABLES, VISHNU 

In [57]:
from collections import defaultdict, Counter
import json
import numpy as np
import time
import mygrad as mg
import gensim
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
%matplotlib inline

import re, string
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))

In [58]:
class Model:
    def __init__(self):
        with open(r'./captions_train2014.json') as json_file:
            self.data = json.load(json_file)
            
        self.caption_ids_to_captions = {cap['id']: cap['caption'] for cap in self.data['annotations']}
        self.image_id_to_caption_id = self.generate_image_to_caption_id()
        
        self.img_ids = tuple(cap['image_id'] for cap in self.data['annotations'])
        self.caption_ids = tuple(cap['id'] for cap in self.data['annotations'])
        self.captions = tuple(cap['caption'] for cap in self.data['annotations'])
        
        path = r"./glove.6B.50d.txt.w2v"
        self.glove = KeyedVectors.load_word2vec_format(path, binary=False)        
    
    def generate_image_to_caption_id(self):
        image_id_to_caption_ids_dict = {}
        
        for cap_dict in self.data['annotations']:
            if cap_dict['image_id'] in image_id_to_caption_ids_dict:
                image_id_to_caption_ids_dict[cap_dict['image_id']].append(cap_dict['id'])
            else:
                image_id_to_caption_ids_dict[cap_dict['image_id']] = [cap_dict['id']]
                
        return image_id_to_caption_ids_dict
        
    def tokenize(self, corpus):
        return punc_regex.sub('', corpus).lower().split()

    def to_df(self, captions):
        
        counter = Counter()
        for caption in captions:
            counter.update(set(self.tokenize(caption))
        return dict(counter)
    
    
    def to_idf(self, captions):
        """ 
        Given the vocabulary, and the word-counts for each document, computes
        the inverse document frequency (IDF) for each term in the vocabulary.

        Parameters
        ----------
        vocab : Sequence[str]
            Ordered list of words that we care about.

        counters : Iterable[collections.Counter]
            The word -> count mapping for each document.

        Returns
        -------
        numpy.ndarray
            An array whose entries correspond to those in `vocab`, storing
            the IDF for each term `t`: 
                               log10(N / nt)
            Where `N` is the number of documents, and `nt` is the number of 
            documents in which the term `t` occurs.
        """
        vishnu = self.to_df(captions)
        return {word : np.log10(len(captions)/cnt + 1) for word, cnt in vishnu.items()}
        
    def caption_to_emb(self, caption, idfs):
        vishnu = sum(self.glove[word] * idfs[word] for word in self.tokenize(caption) if word in self.glove)
        return vishnu.shape
        # remember to normalize vishnu :) go vishnu !!!
    
    def caption_id_to_emb(self):
        idfs = self.to_idf(self.captions)
        return {caption_id : self.caption_to_emb(self.caption_ids_to_captions[caption_id], idfs) for caption_id in self.caption_ids}

In [59]:
model = Model()

In [60]:
model.caption_id_to_emb()

KeyError: "word 'blacksplash' not in vocabulary"